## Dealing with APIs in Python

APIs are everywhere. Learning how to consume them is really important, and in this notebook I'll go through the exploratory process: from learning how to consume APIs, to storing data, and creating real-world solutions.

I'll start by learning the syntax, data types, and nuances by consuming a weather API.

In [3]:
#Importing requests mosule. required for API calls
import requests
#Importing json module. required for parsing JSON data
import json

### First Steps

I'll start from the basics. I'll call a free API that doesn't need a key to get the weather from my city and store the results in a JSON object.

Next, I will set an endpoint and ask the API about the weather in a specific location (Cartago, Valle in this case).

In [4]:
# Example: Get weather for Cartago, Valle del Cauca, Colombia
url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": 4.761190,
    "longitude": -75.899277,
    "current_weather": True,
    "hourly": "relative_humidity_2m"
}

response = requests.get(url, params=params)
data = response.json()
print(data)

{'latitude': 4.75, 'longitude': -75.875, 'generationtime_ms': 0.06747245788574219, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 938.0, 'current_weather_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature': '°C', 'windspeed': 'km/h', 'winddirection': '°', 'is_day': '', 'weathercode': 'wmo code'}, 'current_weather': {'time': '2025-06-25T04:00', 'interval': 900, 'temperature': 19.4, 'windspeed': 4.3, 'winddirection': 275, 'is_day': 0, 'weathercode': 3}, 'hourly_units': {'time': 'iso8601', 'relative_humidity_2m': '%'}, 'hourly': {'time': ['2025-06-25T00:00', '2025-06-25T01:00', '2025-06-25T02:00', '2025-06-25T03:00', '2025-06-25T04:00', '2025-06-25T05:00', '2025-06-25T06:00', '2025-06-25T07:00', '2025-06-25T08:00', '2025-06-25T09:00', '2025-06-25T10:00', '2025-06-25T11:00', '2025-06-25T12:00', '2025-06-25T13:00', '2025-06-25T14:00', '2025-06-25T15:00', '2025-06-25T16:00', '2025-06-25T17:00', '2025-06-25T18:00', '2025-06-25T19:00', '20

Now let's get some real and readable information. As current_weather is another dictionary inside the JSON, let's do some proper calls to it.

current_weather has:
- time
- interval
- temperature
- windspeed
- winddirection
- is_day
- weathercode

In [5]:
current = data['current_weather']
print(f"Temperature: {current['temperature']}°C")
print(f"Wind Speed: {current['windspeed']} km/h")
print(f"Weather Code: {current['weathercode']}")

#Humedad realtiva

current_humidity = data['hourly']['relative_humidity_2m'][0]
print(f"Humedad relativa: {current_humidity}%")

Temperature: 19.4°C
Wind Speed: 4.3 km/h
Weather Code: 3
Humedad relativa: 80%


## What's Next?

I had the idea of requesting the weather from an API, but a user rarely knows the coordinates of their city. Now that I know how to consume an API, I will use Google Maps Geocoding services.

The idea is simple: create a Google Cloud account, get an API key, set the endpoint, understand how the output JSON is structured, and start learning some basic HTTPS responses to make validations.

In [8]:

def get_geocode(city):
    """Get the cordinates of the requested city"""

    url_maps_api ="https://maps.googleapis.com/maps/api/geocode/json"
    key_maps = google_maps_api_key

    #Instancing the parameters to be sent to the API
    params_maps = {
        "address": city,
        "key": key_maps
    }

    #Making the request to the API
    response_maps = requests.get(url_maps_api, params=params_maps)

    #Criteria to check if the request was successful
    #If the status code is 200, it means the request was successful
    if response_maps.status_code == 200:
        data_maps = response_maps.json()

        if data_maps['status'] == 'OK':
            #print(data_maps['results'][0]['geometry'])
            longitude = data_maps['results'][0]['geometry']['location']['lng']
            latitude = data_maps['results'][0]['geometry']['location']['lat']
            return {"latitude": latitude, "longitude": longitude}
        else:
            print(f"Error: No results found for the specified city.")
            return None
    else:
        print(f"Error: Unable to connect to the API. Status code: {response_maps.status_code}")
        return None
        

#Instancing the function to get the geocode of Cartago, Colombia
geocode = get_geocode("Cartago, Colombia")


In [9]:
print(geocode)

{'latitude': 4.747221199999999, 'longitude': -75.91162890000001}


Now the sky is the limit. Just by understanding how JSON files work and connecting to APIs, I can minimize the hassle and get some professional tasks done.

What if I build a Dash App using a more robust weather service?

I could create a graphical interface so that the user can type a city and get the weather info. Nowadays that's really common, but it doesn't take away from the impressiveness if it's just you and a couple of lines of code.

In [47]:

def get_weather(city):
    """Get the main weather conditions for the requested city"""

    #Setting the endpoint and API key for  the Open weather API
    url_weather_api = "https://api.openweathermap.org/data/2.5/weather"
    key_weather = open_weather_api_key

    #Getting the geocode of the city
    geocode = get_geocode(city)
    if not geocode:
        return f"No se encontró la cuidad requesrida: {city}\n verifique la ortografía o intente con otra ciudad."

    #Instancing the parameters to be sent to the API
    params_weather = {
        "lat": geocode["latitude"],
        "lon": geocode["longitude"],
        "appid": key_weather,
        "units": "metric",  # Metric units for temperature in Celsius
    }

    #Making the request to the API
    response_weather = requests.get(url_weather_api, params=params_weather)

    #Criteria to check if the request was successful
    if response_weather.status_code == 200:
        data_weather = response_weather.json()

        #Here goes the code for the data extraction. Keys will depend on the selected API
        #Everything will be set to booleans by now and adjusted later
        temperature = data_weather['main']['temp']
        wind_speed = data_weather['wind']['speed']
        relative_humidity = data_weather['main']['humidity']
        thermal_sensation = data_weather['main']['feels_like']
        precipitation = data_weather['rain']['1h'] if 'rain' in data_weather and '1h' in data_weather['rain'] else 'No Data'
        return {
            "temperature": temperature,
            "wind_speed": wind_speed,
            "realtive_humidity": relative_humidity,
            "thermal_sensation": thermal_sensation,
            "precipitation": precipitation
        }
        
    else:
        print(f"Error: Unable to connect to the API. Status code: {response_weather.status_code}")
        return None
    
#Instancing the function to get the weather of Cartago, Colombia
weather = get_weather("Cartago, Colombia")
print(weather)

{'temperature': 15.35, 'wind_speed': 0.54, 'realtive_humidity': 87, 'thermal_sensation': 15.21, 'precipitation': 'No Data'}


## And what about security?

API keys are sensitive information that can cause a lot of trouble if they land in the wrong hands.

How to manage them?

Fortunately, we have environment variables that can help us set up API keys with much more confidence. Let's see an example.

In [22]:
#Importinf the os module, crucial for evironment variables
#it is important to install the python-dotenv package to use .env files
import os
from dotenv import load_dotenv

In [23]:
#loading the environment variables from the .env file
load_dotenv()

google_maps_api_key = os.getenv("GOOGLE_MAPS_API_KEY")
open_weather_api_key = os.getenv("OPEN_WEATHER_API_KEY") 

## Now a complete version of this App should look like this

In [55]:
def get_geocode(city)-> dict:
    """Get the cordinates of the requested city, 
    returns a dictionary with latitude and longitude"""

    url_maps_api ="https://maps.googleapis.com/maps/api/geocode/json"
    key_maps = google_maps_api_key

    #Instancing the parameters to be sent to the API
    params_maps = {
        "address": city,
        "key": key_maps
    }

    #Making the request to the API
    response_maps = requests.get(url_maps_api, params=params_maps)

    #Criteria to check if the request was successful
    #If the status code is 200, it means the request was successful
    if response_maps.status_code == 200:
        data_maps = response_maps.json()

        if data_maps['status'] == 'OK':
            #print(data_maps['results'][0]['geometry'])
            longitude = data_maps['results'][0]['geometry']['location']['lng']
            latitude = data_maps['results'][0]['geometry']['location']['lat']
            return {"latitude": latitude, "longitude": longitude}
        else:
            print(f"Error: No results found for the specified city.")
            return None
    else:
        print(f"Error: Unable to connect to the API. Status code: {response_maps.status_code}")
        return None

def wind_compass_direction(deg: float) -> str:
    directions = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
    index = int((deg + 22.5) // 45) % 8
    return directions[index]

def get_weather(city)-> dict:
    """Get the main weather conditions for the requested city
    Returns a dictionary with the main weather conditions"""

    #Setting the endpoint and API key for  the Open weather API
    url_weather_api = "https://api.openweathermap.org/data/2.5/weather"
    key_weather = open_weather_api_key

    #Getting the geocode of the city
    geocode = get_geocode(city)
    if not geocode:
        return f"No se encontró la cuidad requesrida: {city}\n verifique la ortografía o intente con otra ciudad."

    #Instancing the parameters to be sent to the API
    params_weather = {
        "lat": geocode["latitude"],
        "lon": geocode["longitude"],
        "appid": key_weather,
        "units": "metric",  # Metric units for temperature in Celsius
    }

    #Making the request to the API
    response_weather = requests.get(url_weather_api, params=params_weather)

    #Criteria to check if the request was successful
    if response_weather.status_code == 200:
        data_weather = response_weather.json()

        #Here goes the code for the data extraction. Keys will depend on the selected API
        #Everything will be set to booleans by now and adjusted later
        temperature = data_weather['main']['temp']
        wind_speed = data_weather['wind']['speed']
        wind_deg = data_weather['wind']['deg']
        wind_direction = wind_compass_direction(wind_deg)
        relative_humidity = data_weather['main']['humidity']
        thermal_sensation = data_weather['main']['feels_like']
        precipitation = data_weather['rain']['1h'] if 'rain' in data_weather and '1h' in data_weather['rain'] else 'No Data'
        #It's important to note that the precipitation key may not always be present in the response, so we check for its existence
        return {
            "temperature": temperature,
            "wind_speed": wind_speed,
            "wind_direction": wind_direction,
            "realtive_humidity": relative_humidity,
            "thermal_sensation": thermal_sensation,
            "precipitation": precipitation
        }
        
    else:
        print(f"Error: Unable to connect to the API. Status code: {response_weather.status_code}")
        return None
    
#Instancing the function to get the weather of Cartago, Colombia
weather = get_weather("Cartago, Colombia")
print(f"Temperature: {weather['temperature']}°C \n"
      f"Thermal Sensation: {weather['thermal_sensation']}°C \n" 
      f"Wind Speed: {round(weather['wind_speed'] * 3.6, 2)} km/h \n" 
      f"Wind Direction: {weather['wind_direction']} \n"
      f"Relative Humidity: {weather['realtive_humidity']}% \n" 
      f"Precipitation: {weather['precipitation']}"
)



Temperature: 15.37°C 
Thermal Sensation: 15.26°C 
Wind Speed: 1.94 km/h 
Wind Direction: N 
Relative Humidity: 88% 
Precipitation: No Data


## Final Thoughts

This is the basis for a functional app that I will be implementing in an upcoming notebook in order to integrate a graphic interface, handle request quotas, and upload to a server.

- It is important to note that Google Maps has its own library for API usage, which makes it much simpler to use the different services they offer. However, the purpose of this notebook was to explore the "hustle" of coding it by yourself.